# Part 1 of Assignment

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude value

!conda install --yes lxml=4.3.0 python=3.6.7
import lxml.html as lh

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

<p>Use the Notebook to build the code to scrape the following Wikipedia page, <a>https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. </p>

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[288, 288, 288]

Perfect! This shows that each of our 3 columns has exactly 288 values.

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


<p>Remove the special character <b>\n</b> from the row values</p>

In [9]:
df=df.replace('\n','',regex=True)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

Rename the columns.
<p>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</p>

In [12]:
df.rename(columns = {'Neighbourhood\n': 'Neighborhood', 'Postcode': 'PostalCode'}, inplace = True)

In [13]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<p>Only process the cells that have an assigned borough. Ignore cells with a borough that is <b>Not assigned.</b></p>

In [15]:
df_not_assigned = df[df['Borough'] == 'Not assigned']
df_not_assigned.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
9,M8A,Not assigned,Not assigned
13,M2B,Not assigned,Not assigned
20,M7B,Not assigned,Not assigned


In [16]:
df.drop(df_not_assigned.index, axis = 0, inplace = True)

In [17]:
df.reset_index(inplace=True)

In [18]:
df.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


In [19]:
df.drop(['index'], axis = 1, inplace = True)

In [20]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<p>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</p>

In [21]:
df_group = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df_group

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<p>If a cell has a borough but a <b>Not assigned</b> neighborhood, then the neighborhood will be the same as the borough. So for the <b>9th</b> cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be <b>Queen's Park</b>.</p>

In [22]:
df_group.Neighborhood.replace('Not assigned',df_group.Borough,inplace=True)

In [23]:
df_group

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<p>In the last cell of your notebook, use the <b>.shape</b> method to print the number of rows of your dataframe.</p>

In [24]:
df_group.shape

(103, 3)

# Part 2 of assignment

In [25]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [26]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
df_coordinates.shape

(103, 3)

Merge df_group data frame and df_coordinates data frame

In [28]:
#Rename Postal Code to PostalCode in df_coordinates data frame
df_coordinates.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
print(df_coordinates.columns)

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')


In [29]:
df = pd.merge(df_group.set_index("PostalCode"),df_coordinates.set_index("PostalCode"), right_index=True, left_index=True).reset_index()

In [30]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 of assignment

<p>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.</p>

In [31]:
#Let's see how many boroughs and neighborhoods are there in the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df['Borough'].unique()), df.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


In [32]:
print('The Boroughs are', df['Borough'].unique())

The Boroughs are ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


Let's create a map using folium that contains all of the boroughs of Toronto, Canada

In [33]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [34]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Let's see how many Neighborhoods are there in each Borough, and sort them in descending order

In [35]:
df.groupby(['Borough']).size().sort_values(ascending = False)

Borough
North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Queen's Park         1
Mississauga          1
dtype: int64

Since <b>North York</b> has the highest number of neighborhoods, let's perform further analysis on that borough.

In [36]:
north_york_df = df[df['Borough'] == 'North York'].reset_index(drop = True)
north_york_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


### Foursquare credentials

In [37]:
CLIENT_ID = 'PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN'
CLIENT_SECRET = 'TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO'
VERSION = '20180605'

print('Client id:{}\nClient Secret:{}\nVersion:{}'.format(CLIENT_ID, CLIENT_SECRET, VERSION))

Client id:PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN
Client Secret:TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO
Version:20180605


### North York Map

In [38]:
address = 'North York, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [39]:
north_york_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(north_york_df['Latitude'], north_york_df['Longitude'], north_york_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(north_york_map)  
    
north_york_map

### Foursquare: Explore Victoria Village
<p>Let's explore top 100 venues in Victoria Village, as it is close to Beach :)</p>

In [40]:
north_york_df.index[north_york_df['Neighborhood'] == 'Victoria Village']

Int64Index([17], dtype='int64')

In [41]:
vv_lat = north_york_df.loc[17, 'Latitude']
vv_long = north_york_df.loc[17, 'Longitude']

print('The latitude and longitude vaues of Victoria Village are {}, {}'.format(vv_lat, vv_long))

The latitude and longitude vaues of Victoria Village are 43.725882299999995, -79.31557159999998


In [42]:
limit = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, vv_lat, vv_long, VERSION, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN&client_secret=TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO&ll=43.725882299999995,-79.31557159999998&v=20180605&limit=100'

In [43]:
#Send the results to a json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d42f7da5d891b00395c508c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 3171,
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 168,
  'suggestedBounds': {'ne': {'lat': 43.755165523051815,
    'lng': -79.27589494412564},
   'sw': {'lat': 43.70343108128725, 'lng': -79.35075850726554}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c00004ff61ea5933223eb13',
       'name': 'Aga Khan Museum',
       'location': {'address': '77 Wynford Dr',
        'crossStreet': 'at Don Valley Pkwy',
        'lat': 43.725105344569876,
        'lng': -79.33207588784165,
        'labeledLa

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Aga Khan Museum,History Museum,43.725105,-79.332076
1,Sultan Of Samosas,Indian Restaurant,43.718823,-79.304350
2,Jatujak,Thai Restaurant,43.736208,-79.307668
3,Diwan,Middle Eastern Restaurant,43.725002,-79.332019
4,Victoria Village Arena,Hockey Arena,43.723481,-79.315635


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [46]:
vv_map = folium.Map(location = [vv_lat, vv_long], zoom_start = 15)

for name, lat, lng in zip(nearby_venues['name'], nearby_venues['lat'], nearby_venues['lng']):
    label = folium.Popup(name, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(vv_map)

vv_map

Let's <b>search</b> for hotels for stay in Victoria Village.

In [47]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query=hotel'.format(CLIENT_ID, CLIENT_SECRET,VERSION,vv_lat,vv_long)
url

'https://api.foursquare.com/v2/venues/search?client_id=PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN&client_secret=TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO&v=20180605&ll=43.725882299999995,-79.31557159999998&query=hotel'

In [48]:
h_results = requests.get(url).json()
h_results

{'meta': {'code': 200, 'requestId': '5d42f7dd2b274a002cea1d6f'},
 'response': {'venues': [{'id': '4b5cb5a4f964a520ca3f29e3',
    'name': 'Toronto Don Valley Hotel and Suites',
    'location': {'address': '175 Wynford Drive',
     'lat': 43.724853496431805,
     'lng': -79.32803128922818,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.724853496431805,
       'lng': -79.32803128922818}],
     'distance': 1008,
     'postalCode': 'M3C 1J3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['175 Wynford Drive',
      'Toronto ON M3C 1J3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '62416033'},
    'referralId': 'v-1564669917',
    'hasPerk': False},
  

Let's stay in the 1st hotel that showed up.

In [49]:
#Display the name and address of the 1st hotel
print('The {} is located at {}, with coordinates of [{}, {}]'.format(h_results['response']['venues'][0]['name'], h_results['response']['venues'][0]['location']['address'], h_results['response']['venues'][0]['location']['lat'], h_results['response']['venues'][0]['location']['lng']))

The Toronto Don Valley Hotel and Suites is located at 175 Wynford Drive, with coordinates of [43.724853496431805, -79.32803128922818]


### Explore the trending venues near the Toronto Don Valley Hotel and Suites

In [50]:
hlat = h_results['response']['venues'][0]['location']['lat']
hlng = h_results['response']['venues'][0]['location']['lng']

url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID,CLIENT_SECRET,hlat,hlng,VERSION)
url

'https://api.foursquare.com/v2/venues/trending?client_id=PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN&client_secret=TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO&ll=43.724853496431805,-79.32803128922818&v=20180605'

Let's get the json results

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d42f7dd66dc060025e2c0bc'},
 'response': {'venues': []}}

Very unfortunate! There are no trending spots near our hotel.
<p>Let's check for any popular hotspots.</p>

In [52]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID,CLIENT_SECRET,hlat,hlng,VERSION)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PDJQGSJOJWKUD3K1HTFN3RQJXW3PLPPIZPH0QQC4GDSMJESN&client_secret=TZNMINK4CSRLYVPL0UQCG25EUFOF3ZD2TTQIGGHPAJEVX4QO&ll=43.724853496431805,-79.32803128922818&v=20180605'

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d42f7de6bdee6311a387bbf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 3255,
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 160,
  'suggestedBounds': {'ne': {'lat': 43.74354020196675,
    'lng': -79.30165090904613},
   'sw': {'lat': 43.706123605608326, 'lng': -79.36102774274936}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c00004ff61ea5933223eb13',
       'name': 'Aga Khan Museum',
       'location': {'address': '77 Wynford Dr',
        'crossStreet': 'at Don Valley Pkwy',
        'lat': 43.725105344569876,
        'lng': -79.33207588784165,
        'labeledLa

In [54]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_hotel_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_hotel_venues = nearby_hotel_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_hotel_venues['venue.categories'] = nearby_hotel_venues.apply(get_category_type, axis=1)

# clean columns
nearby_hotel_venues.columns = [col.split(".")[-1] for col in nearby_hotel_venues.columns]

nearby_hotel_venues.head()

,name,categories,lat,lng
0,Aga Khan Museum,History Museum,43.725105,-79.332076
1,Diwan,Middle Eastern Restaurant,43.725002,-79.332019
2,Japanese Canadian Cultural Centre,General Entertainment,43.726429,-79.334971
3,Kaiseki Yu-Zen Hashimoto,Japanese Restaurant,43.720931,-79.337185
4,Aroma Espresso Bar,Café,43.734400,-79.345204


Let's visualize the hotspots in a map using Folium.

In [55]:
hotel_venues_map = folium.Map(location = [hlat, hlng], zoom_start = 12)

folium.features.CircleMarker(
        [hlat, hlng],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(hotel_venues_map)

for lat, lng, name in zip(nearby_hotel_venues['lat'], nearby_hotel_venues['lng'], nearby_hotel_venues['name']):
    label = folium.Popup(name, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(hotel_venues_map)
    
hotel_venues_map

#### Let's create a function to repeat the same process to all the neighborhoods in North York

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
northyork_venues = getNearbyVenues(names=north_york_df['Neighborhood'],
                                   latitudes=north_york_df['Latitude'],
                                   longitudes=north_york_df['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [58]:
#Let's check the size of the dataframe
print(northyork_venues.shape)
northyork_venues.head()

(199, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [59]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto, Downsview East",2,2,2,2,2,2
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,4,4,4,4,4,4
Downsview West,6,6,6,6,6,6
"Downsview, North Park, Upwood Park",3,3,3,3,3,3


In [60]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 95 uniques categories.


### Normalize and Analyze each neighborhood

In [61]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Financial or Legal Service,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [62]:
northyork_onehot.shape

(199, 96)

In [63]:
#Group the data by neighborhood by taking mean.
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Financial or Legal Service,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.052632,0.0,0.000000,0.000000,0.000000,0.0,0.052632,0.052632,0.000000,0.000000,0.052632,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.00,0.0,0.000000,0.0,0.052632,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.043478,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.043478,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.043478,0.000000,0.086957,0.000000,0.086957,0.043478,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.086957,0.000000,0.00,0.0,0.043478,0.0,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [64]:
northyork_grouped.shape

(22, 96)

#### Let's print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1              Deli / Bodega  0.05
2                Bridal Shop  0.05
3  Middle Eastern Restaurant  0.05
4                      Diner  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Pizza Place  0.09
2           Juice Bar  0.09
3         Coffee Shop  0.09
4    Greek Restaurant  0.04


----CFB Toronto, Downsview East----
                venue  freq
0             Airport   0.5
1                Park   0.5
2    Greek Restaurant   0.0
3       Movie Theater   0.0
4  Miscellaneous Shop   0.0


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1        Baseball Field   0.2


<b>Let's convert that into a data frame</b>

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Deli / Bodega,Diner
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store
2,"Bedford Park, Lawrence Manor East",Juice Bar,Coffee Shop,Italian Restaurant,Pizza Place,Sandwich Place
3,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Event Space,Cosmetics Shop
4,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Café


### Clustering

In [67]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 0, 0, 3, 1, 0, 0, 4], dtype=int32)

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = north_york_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Food Court,Movie Theater,Burger Joint
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN


Since there's one NaN value, let's remove that from the data.

In [69]:
northyork_merged.dropna(subset = ['Cluster Labels'], axis = 0, inplace = True)

In [70]:
northyork_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Food Court,Movie Theater,Burger Joint
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store
5,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Coffee Shop,Ramen Restaurant,Sandwich Place,Café,Middle Eastern Restaurant
6,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Convenience Store,Bank,Park,Event Space,Cosmetics Shop
7,M2R,North York,Willowdale West,43.782736,-79.442259,1.0,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Discount Store
8,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Empanada Restaurant,Convenience Store
9,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Café
10,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1.0,Coffee Shop,Asian Restaurant,Gym,Beer Store,Sporting Goods Shop
11,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,1.0,Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Deli / Bodega,Diner


In [71]:
northyork_merged['Cluster Labels'] = northyork_merged['Cluster Labels'].astype('int')
northyork_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Coffee Shop,Food Court,Movie Theater,Burger Joint
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store
5,M2N,North York,Willowdale South,43.770120,-79.408493,1,Coffee Shop,Ramen Restaurant,Sandwich Place,Café,Middle Eastern Restaurant
6,M2P,North York,York Mills West,43.752758,-79.400049,0,Convenience Store,Bank,Park,Event Space,Cosmetics Shop


<b>Visualize the clusters</b>

In [72]:
# create map
northyork_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, hood, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(hood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(northyork_map_clusters)
       
northyork_map_clusters

<b>Cluster 1</b>

In [73]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,North York,0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store
6,North York,0,Convenience Store,Bank,Park,Event Space,Cosmetics Shop
8,North York,0,Park,Food & Drink Shop,Fast Food Restaurant,Empanada Restaurant,Convenience Store
9,North York,0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Café
13,North York,0,Airport,Park,Women's Store,Event Space,Cosmetics Shop
14,North York,0,Grocery Store,Shopping Mall,Hotel,Bank,Park
20,North York,0,Metro Station,Pub,Japanese Restaurant,Park,Women's Store
21,North York,0,Construction & Landscaping,Bakery,Park,Event Space,Cosmetics Shop


<b>Cluster 2</b>

In [74]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Construction & Landscaping
1,North York,1,Clothing Store,Coffee Shop,Food Court,Movie Theater,Burger Joint
5,North York,1,Coffee Shop,Ramen Restaurant,Sandwich Place,Café,Middle Eastern Restaurant
7,North York,1,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Discount Store
10,North York,1,Coffee Shop,Asian Restaurant,Gym,Beer Store,Sporting Goods Shop
11,North York,1,Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Deli / Bodega,Diner
12,North York,1,Falafel Restaurant,Furniture / Home Store,Coffee Shop,Caribbean Restaurant,Bar
16,North York,1,Coffee Shop,Grocery Store,Discount Store,Liquor Store,Women's Store
17,North York,1,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Financial or Legal Service
18,North York,1,Juice Bar,Coffee Shop,Italian Restaurant,Pizza Place,Sandwich Place


<b>Cluster 3</b>

In [75]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,North York,2,Empanada Restaurant,Women's Store,Event Space,Cosmetics Shop,Deli / Bodega


<b>Cluster 4</b>

In [76]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,North York,3,Food Truck,Home Service,Baseball Field,Women's Store,Event Space


<b>Cluster 5</b>

In [77]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,North York,4,Baseball Field,Construction & Landscaping,Women's Store,Event Space,Cosmetics Shop
